#  KONBRIEFING


## Importación de librerías.

In [63]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import country_converter as coco
import matplotlib.pyplot as plt
import re
import hashlib
from urllib.parse import urlparse
import os
import json
from sklearn.feature_extraction.text import TfidfVectorizer


## Importación e información del Dataframe.

In [64]:
df = pd.read_csv("data/KONBRIEFING.csv")

In [65]:
df.head()

,date,title,description,references,date_uploaded
0,April 2024,Facebook page of the local branch of a politic...,"SPÖ Müllendorf - Müllendorf, Burgenland, Austria","[{""title"":""Hackerangriff auf Facebookseite der...",2024-04-29
1,"April 25, 2024",Cyber attack on a construction company in Germany,"Max Wild GmbH - Berkheim, Baden-Württemberg, G...","[{""title"":""Cyberangriff auf Max Wild GmbH"",""ur...",2024-04-29
2,"April 25, 2024",Cyber attack on a city government in France,"Ville de Gravelines - Gravelines, Hauts-de-Fra...","[{""title"":""Notre ville est actuellement victim...",2024-04-29
3,"April 25, 2024",Cyber attack on a traffic management system in...,"KC Scout - Kansas City, Missouri, USA (Jackson...","[{""title"":""KC Scout Alert"",""url"":""https://www....",2024-04-29
4,"April 24, 2024",Mail account of a city government in Belgium h...,"Deinze, Flemish Region, Belgium","[{""title"":""Stadsdiensten van Deinze geplaagd d...",2024-04-29


In [66]:
df.columns

Index(['date', 'title', 'description', 'references', 'date_uploaded'], dtype='object')

A continuación, se incluye una breve explicación sobre la información que aporta cada columna.
- **Date**: Fecha del ataque (aunque algunos valores están incompletos).
- **Title**: Título del incidente.
- **Description**: Descripción del incidente, que puede incluir la víctima o el sector.
- **References**: Fuentes de referencia para el incidente.
- **Date Uploaded**: Fecha de subida del incidente al sistema.

In [67]:
df.describe(include='all')

,date,title,description,references,date_uploaded
count,67310,67326,67358,67358,67358
unique,1129,4027,4030,4053,16
top,"May 31, 2023",Cyber attack on a university of applied scienc...,? - USA,[],2024-04-29
freq,6624,218,1296,2803,4392


In [68]:
df.dtypes

date             object
title            object
description      object
references       object
date_uploaded    object
dtype: object

## Limpieza de datos.
En un análisis de ciberseguridad, las columnas más importantes son **Date** (fecha del ataque), **Title** (título del incidente) y **Description** (descripción del ataque), ya que proporcionan la información necesaria para identificar patrones temporales, tipos de ataques y sectores afectados. Estas son fundamentales para estudios sobre tendencias, como el aumento de incidentes en ciertos periodos o vulnerabilidades en sectores específicos. Por otro lado, las columnas **Date uploaded** y **References** son menos relevantes para el análisis de patrones, ya que solo indican cuándo se documentó el incidente y las fuentes, aunque pueden ser útiles para validar la información.

In [69]:
df.drop(columns=['date_uploaded'], inplace=True)

El proceso consiste en convertir las cadenas de la columna, que pueden representar listas de diccionarios, en estructuras adecuadas para extraer la información relevante.

Primero, las cadenas se convierten en listas de diccionarios, manejando posibles errores de formato mediante excepciones para evitar fallos. Cuando la conversión es exitosa, se extraen los valores de las claves `title` y `url` de los diccionarios dentro de la lista, asignando valores predeterminados como `None` si no se encuentran datos. Esto permite crear dos nuevas columnas: **References title** y **References URL**.

### Manejo de filas duplicadas.
En caso de haber filas duplicadas, deberemos de eliminar estas para que no afecten al análisis posterior.

In [70]:
df.duplicated().sum() # Columnas duplicadas

62970

Los duplicados en el conjunto de datos representan registros que contienen información redundante o repetitiva, lo que puede sesgar los análisis y afectar la calidad de un modelo de Machine Learning. En este caso, se consideran duplicados aquellos registros que comparten el mismo título, descripción y referencias, ya que estas columnas capturan la esencia del contenido del dato. La columna **Date Uploaded**, que refleja el momento en que los datos fueron cargados al sistema, no será relevante para definir duplicados porque no aporta información directa sobre los ciberataques en sí. Al eliminar los duplicados, aseguramos que el modelo trabaje con datos únicos y representativos, evitando sesgos y optimizando la eficiencia del análisis.

In [71]:
df.drop_duplicates(inplace = True)

In [72]:
df.shape

(4388, 4)

### Manejo de filas nulas.

In [73]:
df.isnull().sum()

date           3
title          2
description    0
references     0
dtype: int64

In [74]:
df.dropna(inplace=True)

### Selección de variables categóricas y continuas.
Para realizar correctamente un ánalisis de datos debemos de distinguir entre las variables categóricas, variables que representan diferentes categorías o grupos, y continuas, variables que pueden tomar cualquier valor dentro de un rango determinado.

In [75]:
v_continuas = []
v_categoricas = []
for i in df.columns:
    if df[i].nunique() > 70 or df[i].dtypes in ['float64', 'int64']:
        v_continuas.append(i)
    else:
        v_categoricas.append(i)

print('Variables continuas: {}'.format(', '.join(v_continuas)))
print('Variables categóricas: {}'.format(', '.join(v_categoricas)))

Variables continuas: date, title, description, references
Variables categóricas: 


#### Tratamiento de variables continuas.
Para el modelo de clasificación debemos de pasar todas las columnas a tipo `int`. Por ello, debemos de ver si realizar como en el caso anterior una codificación ordinal o agruparlos.

##### **Date.**

In [76]:
df['date'] = pd.to_datetime(df['date'])

ParserError: Unknown string format: April 2024 ?

### Description

In [128]:
data_cleaned['country'] = data_cleaned['description'].apply(lambda x: x.split(',')[-1].strip() if isinstance(x, str) else None)

attacks_per_country = data_cleaned['country'].value_counts()

print(attacks_per_country)


country
Germany                                                              6067
France                                                               4535
USA                                                                  4248
United Kingdom                                                       2930
Canada                                                               2648
                                                                     ... 
USA (Coffee County)                                                     1
Germany (Kreis Recklinghausen)                                          1
Skanlog - Sweden                                                        1
České noviny (ČN) / Česká tisková kancelář (ČTK) - Czech Republik       1
Germany (Landkreis Biberach)                                            1
Name: count, Length: 929, dtype: int64


In [129]:
def clean_country_name(country):
  if '-' in country:
    cleaned_name = country.split('-')[-1].strip()
  else:
    cleaned_name = re.sub(r"\(.*?\)", "", country).strip()
  return cleaned_name.strip()

  data_cleaned['country_clean'] = data_cleaned['country'].apply(clean_country_name)

In [ ]:
continents_mapping = {
    'Africa': [
        'Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cameroon', 'Central African Republic',
        'Chad', 'Comoros', 'Congo', 'Djibouti', 'Egypt', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Ethiopia', 'Gabon',
        'Gambia', 'Ghana', 'Guinea', 'Guinea-Bissau', 'Ivory Coast', 'Kenya', 'Lesotho', 'Liberia', 'Libya', 'Madagascar',
        'Malawi', 'Mali', 'Mauritania', 'Mauritius', 'Morocco', 'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda',
        'Sao Tome and Principe', 'Senegal', 'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa', 'South Sudan',
        'Sudan', 'Tanzania', 'Togo', 'Tunisia', 'Uganda', 'Zambia', 'Zimbabwe', 'Libya / ليبيا'
    ],
    'Asia': [
        'Afghanistan', 'Armenia', 'Bahrain', 'Bangladesh', 'Bhutan', 'Brunei', 'Cambodia', 'China', 'Cyprus',
        'India', 'Indonesia', 'Iran', 'Iraq', 'Israel', 'Japan', 'Jordan', 'Kazakhstan', 'Kuwait', 'Kyrgyzstan',
        'Laos', 'Lebanon', 'Malaysia', 'Maldives', 'Mongolia', 'Myanmar', 'Nepal', 'North Korea', 'Oman', 'Pakistan',
        'Palestine', 'Philippines', 'Qatar', 'Saudi Arabia', 'Singapore', 'South Korea', 'Sri Lanka', 'Syria', 'Taiwan',
        'Tajikistan', 'Thailand', 'Timor-Leste', 'Turkey', 'Turkmenistan', 'United Arab Emirates', 'Uzbekistan', 'Vietnam', 'Yemen',
        "People's Republic of China", 'Republic of China / Taiwan', 'Jordanien / الأردن', 'Malediven / ދިވެހިރާއްޖޭގެ ޖުމްހޫރިއްޔާ'
    ],
    'Europe': [
        'Albania', 'Andorra', 'Austria', 'Azerbaijan', 'Belarus', 'Belgium', 'Bosnia and Herzegovina', 'Bulgaria',
        'Croatia', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Georgia', 'Germany', 'Greece',
        'Hungary', 'Iceland', 'Ireland', 'Italy', 'Kosovo', 'Latvia', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Malta',
        'Moldova', 'Monaco', 'Montenegro', 'Netherlands', 'North Macedonia', 'Norway', 'Poland', 'Portugal', 'Romania',
        'Russia', 'San Marino', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Ukraine', 'United Kingdom',
        'Vatican City', 'Czech Republik', 'Upper Austria', 'Kingdom of the Netherlands', 'Serbien / Србија', 'UK', 'Isle of Man',
        'Vatican'
    ],
    'North America': [
        'Antigua and Barbuda', 'Bahamas', 'Barbados', 'Belize', 'Canada', 'Costa Rica', 'Cuba', 'Dominica', 'Dominican Republic',
        'El Salvador', 'Grenada', 'Guatemala', 'Haiti', 'Honduras', 'Jamaica', 'Mexico', 'Nicaragua', 'Panama',
        'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Vincent and the Grenadines', 'Trinidad and Tobago', 'United States of America (USA)',
        'USA', 'Grönland', 'Puerto Rico', 'Bermuda', 'United States of America'
    ],
    'South America': [
        'Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Guyana', 'Paraguay', 'Peru', 'Suriname', 'Uruguay', 'Venezuela'
    ],
    'Australia': [
        'Australia', 'Fiji', 'Kiribati', 'Marshall Islands', 'Micronesia', 'Nauru', 'New Zealand', 'Palau', 'Papua New Guinea',
        'Samoa', 'Solomon Islands', 'Tonga', 'Tuvalu', 'Vanuatu', 'Oceania'
    ],
    'Africa, Asia, North America, Europe, South America, Australia': ['Global']
}

In [131]:
def assign_continent(country_clean):
  continents = set()

  for continent, countries_list in continents_mapping.items():
    if country_clean in countries_list:
      continents.add(continent)
  return ', '.join(continents) if continents else 'Sin continente'

data_cleaned['country_clean'] = data_cleaned['country'].apply(clean_country_name)
data_cleaned['continent'] = data_cleaned['country_clean'].apply(assign_continent)

sin_continente_rows = data_cleaned[data_cleaned['continent'] == 'Sin continente']
print(sin_continente_rows)

print(f"Número de filas sin continente: {len(sin_continente_rows)}")


                                                   title  \
3      Cyber attack on a traffic management system in...   
36     Unauthorized access at a telecommunications pr...   
71     Ransomware at a county government in Missouri,...   
78     Cyber attack on a school district in Michigan,...   
85              Cyber attack on a hotel chain in the USA   
...                                                  ...   
67204  Unauthorized access at a benefit plan service ...   
67241  Cyber attack with emails on global furniture s...   
67267  Hackers capture $200 million from the BitMart ...   
67286   Cyberattack on a healthcare provider in Guernsey   
67304  140 million US dollars stolen from crypto plat...   

                                             description  \
3      KC Scout - Kansas City, Missouri, USA (Jackson...   
36     Frontier Communications - Dallas, Texas, USA (...   
71     Jackson County - Kansas City, Missouri, USA (J...   
78     Traverse City Area Public School

In [132]:
data_cleaned.head(10)

,title,description,references,date_uploaded,year,month,day,country,country_clean,continent
0,Facebook page of the local branch of a politic...,"SPÖ Müllendorf - Müllendorf, Burgenland, Austria","[{""title"":""Hackerangriff auf Facebookseite der...",2024-04-29,2024,4,1,Austria,Austria,Europe
1,Cyber attack on a construction company in Germany,"Max Wild GmbH - Berkheim, Baden-Württemberg, G...","[{""title"":""Cyberangriff auf Max Wild GmbH"",""ur...",2024-04-29,2024,4,25,Germany (Landkreis Biberach),Germany,Europe
2,Cyber attack on a city government in France,"Ville de Gravelines - Gravelines, Hauts-de-Fra...","[{""title"":""Notre ville est actuellement victim...",2024-04-29,2024,4,25,France,France,Europe
3,Cyber attack on a traffic management system in...,"KC Scout - Kansas City, Missouri, USA (Jackson...","[{""title"":""KC Scout Alert"",""url"":""https://www....",2024-04-29,2024,4,25,Platte County),Platte County),Sin continente
4,Mail account of a city government in Belgium h...,"Deinze, Flemish Region, Belgium","[{""title"":""Stadsdiensten van Deinze geplaagd d...",2024-04-29,2024,4,24,Belgium,Belgium,Europe
5,A municipality in Argentina loses 19 million p...,"San Agustín, Provincia de Santa Fe, Argentina","[{""title"":""LA COMUNA DE SAN AGUSTÍN VÍCTIMA DE...",2024-04-29,2024,4,1,Argentina,Argentina,South America
6,Cyber attack on a municipality in Canada,"Municipalité La Guadeloupe - La Guadeloupe, Qu...","[{""title"":""Le 23 avril dernier, la Municipalit...",2024-04-29,2024,4,23,Canada,Canada,North America
7,Website of a Czech news agency hacked,České noviny (ČN) / Česká tisková kancelář (ČT...,"[{""title"":""UPOZORNĚNÍ ČTK: Útočník napadl web ...",2024-04-29,2024,4,23,České noviny (ČN) / Česká tisková kancelář (ČT...,Czech Republik,Europe
8,"Cyber attack on a logistics company in Sweden,...",Skanlog - Sweden,"[{""title"":""Hackerangrep kan gi tomme hyller ho...",2024-04-29,2024,4,1,Skanlog - Sweden,Sweden,Europe
9,Cyber attack on a city government in France,"Ville d'Albi - Albi, Occitanie, France","[{""title"":""La Ville d'Albi victime d'une attaq...",2024-04-29,2024,4,22,France,France,Europe


In [133]:
# Identificar las filas donde la columna 'continent' es NaN
df_null_continent = data_cleaned['continent'].isnull()

# Mostrar las filas correspondientes
countries_with_no_continent = data_cleaned[df_null_continent]['country'].unique()

# Mostrar la lista de países con 'continent' como None
print("Lista de países con 'continent' como None:")
print(countries_with_no_continent)


Lista de países con 'continent' como None:
[]


In [134]:
# Contar cuántos valores nulos hay en la columna 'continent'
num_null_continent = data_cleaned['continent'].isnull().sum()

# Mostrar el número de valores nulos
print(f"Cantidad de valores nulos en la columna 'continent': {num_null_continent}")


Cantidad de valores nulos en la columna 'continent': 0


In [135]:
data_cleaned.drop(columns=['country'], inplace=True)
data_cleaned.drop(columns=['country_clean'], inplace=True)
data_cleaned.head(10)

,title,description,references,date_uploaded,year,month,day,continent
0,Facebook page of the local branch of a politic...,"SPÖ Müllendorf - Müllendorf, Burgenland, Austria","[{""title"":""Hackerangriff auf Facebookseite der...",2024-04-29,2024,4,1,Europe
1,Cyber attack on a construction company in Germany,"Max Wild GmbH - Berkheim, Baden-Württemberg, G...","[{""title"":""Cyberangriff auf Max Wild GmbH"",""ur...",2024-04-29,2024,4,25,Europe
2,Cyber attack on a city government in France,"Ville de Gravelines - Gravelines, Hauts-de-Fra...","[{""title"":""Notre ville est actuellement victim...",2024-04-29,2024,4,25,Europe
3,Cyber attack on a traffic management system in...,"KC Scout - Kansas City, Missouri, USA (Jackson...","[{""title"":""KC Scout Alert"",""url"":""https://www....",2024-04-29,2024,4,25,Sin continente
4,Mail account of a city government in Belgium h...,"Deinze, Flemish Region, Belgium","[{""title"":""Stadsdiensten van Deinze geplaagd d...",2024-04-29,2024,4,24,Europe
5,A municipality in Argentina loses 19 million p...,"San Agustín, Provincia de Santa Fe, Argentina","[{""title"":""LA COMUNA DE SAN AGUSTÍN VÍCTIMA DE...",2024-04-29,2024,4,1,South America
6,Cyber attack on a municipality in Canada,"Municipalité La Guadeloupe - La Guadeloupe, Qu...","[{""title"":""Le 23 avril dernier, la Municipalit...",2024-04-29,2024,4,23,North America
7,Website of a Czech news agency hacked,České noviny (ČN) / Česká tisková kancelář (ČT...,"[{""title"":""UPOZORNĚNÍ ČTK: Útočník napadl web ...",2024-04-29,2024,4,23,Europe
8,"Cyber attack on a logistics company in Sweden,...",Skanlog - Sweden,"[{""title"":""Hackerangrep kan gi tomme hyller ho...",2024-04-29,2024,4,1,Europe
9,Cyber attack on a city government in France,"Ville d'Albi - Albi, Occitanie, France","[{""title"":""La Ville d'Albi victime d'une attaq...",2024-04-29,2024,4,22,Europe


### References


In [136]:
def extract_title_url(reference_column):
    titles, urls = [], []

    for ref in reference_column:
        if not ref:
            titles.append("")
            urls.append("")
            continue

        try:

            ref_json = json.loads(ref)


            if isinstance(ref_json, list) and len(ref_json) > 0:
                first_ref = ref_json[0]


                title = first_ref.get("title", "")
                url = first_ref.get("url", "")


                site_name = urlparse(url).netloc.replace('www.', '') if url else ""

                titles.append(title)
                urls.append(site_name)
            else:
                titles.append("")
                urls.append("")
        except (json.JSONDecodeError, IndexError, TypeError) as e:

            titles.append("")
            urls.append("")

    return titles, urls

data_cleaned['title_extracted'], data_cleaned['site_name_extracted'] = extract_title_url(data_cleaned['references'])

data_cleaned.head(10)


,title,description,references,date_uploaded,year,month,day,continent,title_extracted,site_name_extracted
0,Facebook page of the local branch of a politic...,"SPÖ Müllendorf - Müllendorf, Burgenland, Austria","[{""title"":""Hackerangriff auf Facebookseite der...",2024-04-29,2024,4,1,Europe,Hackerangriff auf Facebookseite der SPÖ Müllen...,meinbezirk.at
1,Cyber attack on a construction company in Germany,"Max Wild GmbH - Berkheim, Baden-Württemberg, G...","[{""title"":""Cyberangriff auf Max Wild GmbH"",""ur...",2024-04-29,2024,4,25,Europe,Cyberangriff auf Max Wild GmbH,maxwild.com
2,Cyber attack on a city government in France,"Ville de Gravelines - Gravelines, Hauts-de-Fra...","[{""title"":""Notre ville est actuellement victim...",2024-04-29,2024,4,25,Europe,Notre ville est actuellement victime d’une cyb...,facebook.com
3,Cyber attack on a traffic management system in...,"KC Scout - Kansas City, Missouri, USA (Jackson...","[{""title"":""KC Scout Alert"",""url"":""https://www....",2024-04-29,2024,4,25,Sin continente,KC Scout Alert,facebook.com
4,Mail account of a city government in Belgium h...,"Deinze, Flemish Region, Belgium","[{""title"":""Stadsdiensten van Deinze geplaagd d...",2024-04-29,2024,4,24,Europe,Stadsdiensten van Deinze geplaagd door cyberaa...,nieuwsblad.be
5,A municipality in Argentina loses 19 million p...,"San Agustín, Provincia de Santa Fe, Argentina","[{""title"":""LA COMUNA DE SAN AGUSTÍN VÍCTIMA DE...",2024-04-29,2024,4,1,South America,LA COMUNA DE SAN AGUSTÍN VÍCTIMA DE DELITO CIB...,facebook.com
6,Cyber attack on a municipality in Canada,"Municipalité La Guadeloupe - La Guadeloupe, Qu...","[{""title"":""Le 23 avril dernier, la Municipalit...",2024-04-29,2024,4,23,North America,"Le 23 avril dernier, la Municipalité de La Gua...",facebook.com
7,Website of a Czech news agency hacked,České noviny (ČN) / Česká tisková kancelář (ČT...,"[{""title"":""UPOZORNĚNÍ ČTK: Útočník napadl web ...",2024-04-29,2024,4,23,Europe,UPOZORNĚNÍ ČTK: Útočník napadl web České novin...,ctk.cz
8,"Cyber attack on a logistics company in Sweden,...",Skanlog - Sweden,"[{""title"":""Hackerangrep kan gi tomme hyller ho...",2024-04-29,2024,4,1,Europe,Hackerangrep kan gi tomme hyller hos Systembol...,e24.no
9,Cyber attack on a city government in France,"Ville d'Albi - Albi, Occitanie, France","[{""title"":""La Ville d'Albi victime d'une attaq...",2024-04-29,2024,4,22,Europe,La Ville d'Albi victime d'une attaque informat...,mairie-albi.fr


In [ ]:
data_cleaned.drop(columns=['references'], inplace=True)
data_cleaned.drop(columns=['date_uploaded'], inplace=True)

In [139]:
# Columnas tratadas
data_cleaned.head()

,title,description,year,month,day,continent,title_extracted,site_name_extracted
0,Facebook page of the local branch of a politic...,"SPÖ Müllendorf - Müllendorf, Burgenland, Austria",2024,4,1,Europe,Hackerangriff auf Facebookseite der SPÖ Müllen...,meinbezirk.at
1,Cyber attack on a construction company in Germany,"Max Wild GmbH - Berkheim, Baden-Württemberg, G...",2024,4,25,Europe,Cyberangriff auf Max Wild GmbH,maxwild.com
2,Cyber attack on a city government in France,"Ville de Gravelines - Gravelines, Hauts-de-Fra...",2024,4,25,Europe,Notre ville est actuellement victime d’une cyb...,facebook.com
3,Cyber attack on a traffic management system in...,"KC Scout - Kansas City, Missouri, USA (Jackson...",2024,4,25,Sin continente,KC Scout Alert,facebook.com
4,Mail account of a city government in Belgium h...,"Deinze, Flemish Region, Belgium",2024,4,24,Europe,Stadsdiensten van Deinze geplaagd door cyberaa...,nieuwsblad.be
